In [2]:
# 3_model_training.ipynb

# ---
# 目的: 读取特征 => 构建简单模型 => 训练 => 保存 model.pt
# 注意: 只有2首歌几乎无法训练出有意义模型, 仅演示流程

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

FEATURE_FOLDER = "../data/processed/"
MODEL_PATH = "../saved_models/mlp_chorus_model.pt"

# 1. 加载 feature_index.csv
df = pd.read_csv(os.path.join(FEATURE_FOLDER, "feature_index.csv"))
print(df)

# 2. 伪造标签(仅演示) => 0=非副歌, 1=副歌
#   如果你想做2类分类(副歌/非副歌), 你需要打样本label
#   目前只有2首 => 可能随意
features_list = []
labels_list = []
for idx, row in df.iterrows():
    feat_path = row["feature_path"]
    feats = np.load(feat_path)
    # feats shape (13,) => let's store them
    # demo: assume the first file label=0, second=1
    label = 0 if idx==0 else 1

    features_list.append(feats)
    labels_list.append(label)

X = np.array(features_list, dtype=np.float32)  # shape (N, 13)
y = np.array(labels_list, dtype=np.int64)      # shape (N,)

print("X shape=", X.shape, "y shape=", y.shape)

# 3. 构建简单 MLP
class MLP(nn.Module):
    def __init__(self, input_dim=13, hidden_dim=32, output_dim=2):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = MLP(input_dim=13, hidden_dim=16, output_dim=2)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 4. 训练(极简)
X_torch = torch.from_numpy(X)
y_torch = torch.from_numpy(y)

for epoch in range(50):
    optimizer.zero_grad()
    out = model(X_torch)
    loss = criterion(out, y_torch)
    loss.backward()
    optimizer.step()
    if (epoch+1)%10==0:
        print(f"Epoch {epoch+1}, loss={loss.item():.4f}")

# 5. 保存
os.makedirs("../saved_models", exist_ok=True)
torch.save(model.state_dict(), MODEL_PATH)
print("模型训练完毕, 已保存到", MODEL_PATH)

Epoch [2/10], Loss: 0.0204
Epoch [4/10], Loss: 0.0020
Epoch [6/10], Loss: 0.0004
Epoch [8/10], Loss: 0.0001
Epoch [10/10], Loss: 0.0000
模型训练完成并已保存
